# Challenge State of Data

### Sobre os dados

O questionário foi dividido em 8 partes, e dentro de cada uma das partes temos as perguntas e opções de escolha.

- Parte 1 - Dados demográficos
- Parte 2 - Dados sobre carreira
- Parte 3 - Desafios dos gestores de times de dados
- Parte 4 - Conhecimentos na área de dados
- Parte 5 - Objetivos na área de dados
- Parte 6 - Conhecimentos em Engenharia de Dados/DE
- Parte 7 - Conhecimentos em Análise de Dados/DA
- Parte 8 - Conhecimentos em Ciências de Dados/DS

Cada pergunta é dividida em Parte, Letra da Pergunta, Número da Opção escolhida
- Exemplo: P3a_1 = Parte 3, pergunta (a), opção (1)

## 0. Configuração do ambiente

### 0.1. Bibliotecas

In [1]:
import pandas as pd
import numpy as np

### 0.2. Dados

In [2]:
data = pd.read_csv('State_of_data_2022.csv')

In [3]:
# criar dicionário com a descrição de cada parte

descricao_parte = {
    'parte': ['parte1', 'parte2', 'parte3', 'parte4', 'parte5', 'parte6', 'parte7', 'parte8'],
    'descricao': ['dados demográficos', 
                  'dados sobre carreira', 
                  'desafios dos gestores de times de dados',
                  'conhecimentos na área de dados',
                  'objetivos na área de dados',
                  'conhecimentos em engenharia de dados',
                  'conhecimentos em análise de dados',
                  'conhecimentos em ciência de dados']
}

descricao_parte = pd.DataFrame(descricao_parte)

## 1. Visão geral do dataframe

In [5]:
# número de linhas e de colunas

data.shape

(4271, 353)

In [ ]:
# como esta base tem muitas colunas, vou checar os temas pelas partes

### 1.1. Função para gerar tabelas de descrição para cada parte

In [49]:
# função para criar uma tabela para cada parte, separando o código, a descrição, a pergunta e as opções

def gerar_tabela_descricao(parte):

    df = data.filter(regex = "^\('P" + str(parte), axis=1).columns.to_frame(name = 'lista').reset_index(drop = True)

    separar = df['lista'].str.split(',', n = 1, expand = True)

    df['codigo'] = separar[0]
    df['codigo'] = df['codigo'].str.replace('(', '')
    df['codigo'] = df['codigo'].str.replace("'", '')
    df['codigo'] = df['codigo'].str.strip()

    df['descricao'] = separar[1]
    df['descricao'] = df['descricao'].str.replace(')', '')
    df['descricao'] = df['descricao'].str.replace("'", '')

    df['pergunta'] = df['codigo'].apply(lambda x: x[3])
    df['opcao'] = df['codigo'].apply(lambda x: x[5] if len(x) >= 6 else None)

    df = df.drop(['lista'], axis = 1)

    return df

# deixar cada dataframe gerado disponível no ambiente de variáveis

for i in list(range(1, 9)):

    dataframe_name = f'parte{i}_descricao'
    df = gerar_tabela_descricao(parte = i)
    globals()[dataframe_name] = df

### 1.2. Função para gerar tabelas para cada parte, alterando o nome das colunas para ser apenas o código

In [50]:
# função para criar uma tabela para cada parte, trocando o nome das colunas em forma de código para apenas a tupla

def partes(parte):

    df = data.filter(regex = "^\('P" + str(parte), axis = 1)
    parte_n = eval('parte' + str(parte) + '_descricao')
    df.columns = parte_n['codigo'].tolist()

    return df

# deixar os dataframes no ambiente de variáveis

for i in list(range(1, 9)):
    
    dataframe_name = f'parte{i}'
    df = partes(parte = i)
    globals()[dataframe_name] = df

### 1.3. Função para calcular o percentual de missings de cada coluna

In [81]:
# função para acrescentar uma coluna com o percentual de missings na tabela de descricao

def perc_missings(n_df):

    df_ = eval('parte' + str(n_df) + '_descricao').copy()
    df_['perc_miss'] = df_['codigo'].apply(lambda x: round(eval('parte' + str(n_df))[x].isna().mean() * 100, 2))

    return df_

### 1.4. Análises dos dataframes

In [21]:
# quantas colunas tem em cada parte?

descricao_parte_colunas = descricao_parte.copy()

descricao_parte_colunas['n_colunas'] = descricao_parte_colunas['parte'].apply(lambda x: len(eval(x + '_descricao')))

descricao_parte_colunas['n_perguntas_diferentes'] =  descricao_parte_colunas['parte'].apply(lambda x: eval(x + '_descricao')['pergunta'].nunique())

descricao_parte_colunas

# na forma de lista
# for i in [key for key in list(globals().keys()) if key.startswith('parte')]:
#     print(i, ":", len(eval(i)))

,parte,descricao,n_colunas,n_perguntas_diferentes
0,parte1,dados demográficos,15,12
1,parte2,dados sobre carreira,36,19
2,parte3,desafios dos gestores de times de dados,38,4
3,parte4,conhecimentos na área de dados,100,9
4,parte5,objetivos na área de dados,4,4
5,parte6,conhecimentos em engenharia de dados,59,8
6,parte7,conhecimentos em análise de dados,50,5
7,parte8,conhecimentos em ciência de dados,50,5


## 2. Sanity check

### Parte 1 - Dados demográficos

In [24]:
# visão geral dos dados

parte1.head()

,P1_a,P1_a_1,P1_b,P1_c,P1_d,P1_e,P1_f,P1_g,P1_i,P1_i_1,P1_i_2,P1_j,P1_k,P1_l,P1_m
0,39.0,35-39,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,NaN,True,Distrito Federal (DF),DF,Centro-oeste,False,Sudeste,Pós-graduação,Computação / Engenharia de Software / Sistemas...
1,32.0,30-34,Masculino,Parda,Não,"Sim, acredito que a minha a experiência profis...",Aprovação em processos seletivos/entrevistas,True,Pará (PA),PA,Norte,True,NaN,Graduação/Bacharelado,Outras Engenharias
2,53.0,50-54,Masculino,Branca,Não,NaN,NaN,True,Distrito Federal (DF),DF,Centro-oeste,False,Sul,Pós-graduação,Computação / Engenharia de Software / Sistemas...
3,27.0,25-29,Masculino,Branca,Não,NaN,NaN,True,Minas Gerais (MG),MG,Sudeste,True,NaN,Doutorado ou Phd,Estatística/ Matemática / Matemática Computaci...
4,46.0,45-49,Feminino,Branca,Não,Não acredito que minha experiência profissiona...,NaN,True,Pará (PA),PA,Norte,True,NaN,Pós-graduação,Computação / Engenharia de Software / Sistemas...


In [85]:
# olhar o percentual de missings de cada coluna

perc_missings(n_df = 1)

,codigo,descricao,pergunta,opcao,perc_miss
0,P1_a,Idade,a,None,1.73
1,P1_a_1,Faixa idade,a,1,0.00
2,P1_b,Genero,b,None,0.21
3,P1_c,Cor/raca/etnia,c,None,0.00
4,P1_d,PCD,d,None,0.00
5,P1_e,experiencia_profissional_prejudicada,e,None,48.72
6,P1_f,aspectos_prejudicados,f,None,80.40
7,P1_g,vive_no_brasil,g,None,0.00
8,P1_i,Estado onde mora,i,None,2.69
9,P1_i_1,uf onde mora,i,1,2.69


In [97]:
parte1['P1_a_1'].value_counts().to_frame().reset_index().sort_values('P1_a_1')

,P1_a_1,count
5,17-21,152
3,22-24,449
0,25-29,1398
1,30-34,1095
2,35-39,598
4,40-44,277
6,45-49,145
7,50-54,89
8,55+,68


In [91]:
# qual a distribuição de faixa de idade para quem não informou idade?

parte1[parte1['P1_a'].isna()]['P1_a_1'].value_counts()

P1_a_1
55+      68
17-21     6
Name: count, dtype: int64

In [99]:
# como os nulos de faixa de idade foram preenchidos?

parte1[parte1['P1_a'].isna()].sort_values('P1_a_1').head(20)

,P1_a,P1_a_1,P1_b,P1_c,P1_d,P1_e,P1_f,P1_g,P1_i,P1_i_1,P1_i_2,P1_j,P1_k,P1_l,P1_m
4269,NaN,17-21,Masculino,Branca,Não,NaN,NaN,True,São Paulo (SP),SP,Sudeste,True,NaN,Não tenho graduação formal,NaN
3367,NaN,17-21,Feminino,Parda,Não,Não acredito que minha experiência profissiona...,NaN,True,Bahia (BA),BA,Nordeste,True,NaN,Não tenho graduação formal,NaN
2393,NaN,17-21,Feminino,Branca,Não,Não acredito que minha experiência profissiona...,NaN,True,Bahia (BA),BA,Nordeste,True,NaN,Não tenho graduação formal,NaN
1925,NaN,17-21,Prefiro não informar,Prefiro não informar,Prefiro não informar,"Sim, acredito que minha experiência profission...",Quantidade de oportunidades de emprego/vagas r...,False,NaN,NaN,NaN,NaN,NaN,Prefiro não informar,NaN
4177,NaN,17-21,Masculino,Branca,Não,NaN,NaN,True,Rio Grande do Norte (RN),RN,Nordeste,True,NaN,Não tenho graduação formal,NaN
4086,NaN,17-21,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,NaN,True,Bahia (BA),BA,Nordeste,True,NaN,Estudante de Graduação,Computação / Engenharia de Software / Sistemas...
2436,NaN,55+,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,NaN,True,Bahia (BA),BA,Nordeste,True,NaN,Pós-graduação,Estatística/ Matemática / Matemática Computaci...
2488,NaN,55+,Masculino,Branca,Não,NaN,NaN,True,Minas Gerais (MG),MG,Sudeste,False,Sudeste,Pós-graduação,Economia/ Administração / Contabilidade / Fina...
2508,NaN,55+,Masculino,Branca,Não,NaN,NaN,True,Rio Grande do Sul (RS),RS,Sul,True,NaN,Doutorado ou Phd,Economia/ Administração / Contabilidade / Fina...
2752,NaN,55+,Masculino,Branca,Não,NaN,NaN,True,São Paulo (SP),SP,Sudeste,True,NaN,Graduação/Bacharelado,Outra opção


### Parte 2 - Dados sobre carreira

### Parte 3 - Desafios dos gestores de times de dados

### Parte 4 - Conhecimentos na área de dados

### Parte 5 - Objetivos na área de dados

### Parte 6 - Conhecimentos em engenharia de dados

### Parte 7 - Conhecimentos em análise de dados

### Parte 8 - Conhecimentos em ciências de dados

## 3. Análise exploratória